In [1]:
import pandas as pd

In [3]:
header=['user_id', 'item_id', 'rating']
content_data = pd.read_csv('metaSim.csv', sep=',', names=header)
tag_data = pd.read_csv('tagsSim.csv', sep=',', names=header)
cf_data = pd.read_csv('cf_pre.csv', sep=',', names=header)
svd_data = pd.read_csv('lfm_pre.csv', sep=',', names=header)

In [4]:
header=['user_id', 'item_id', 'rating', 'timestamp']
train_data = pd.read_csv('train.csv', sep=',', names=header)
test_data = pd.read_csv('test.csv', sep=',' , names=header)

In [6]:
r={} #movie by user
content_sim={} #content based
tag_sim={} #based on tag
cf_pre={} #collaborative filtering
svd_pre={} #latent factor

In [9]:
for line in train_data.itertuples():        
    if line[2] in r:
        r[line[2]].update({line[1]: line[3]})
    else:
        r.update({line[2]: {line[1]: line[3]}})

for line in content_data.itertuples():        
    if line[2] in content_sim:
        content_sim[line[2]].update({line[1]: line[3]})
    else:
        content_sim.update({line[2]: {line[1]: line[3]}})

for line in tag_data.itertuples():        
    if line[2] in tag_sim:
        tag_sim[line[2]].update({line[1]: line[3]})
    else:
        tag_sim.update({line[2]: {line[1]: line[3]}})

for line in cf_data.itertuples():        
    if line[2] in cf_pre:
        cf_pre[line[2]].update({line[1]: line[3]})
    else:
        cf_pre.update({line[2]: {line[1]: line[3]}})

for line in svd_data.itertuples():        
    if line[2] in svd_pre:
        svd_pre[line[2]].update({line[1]: line[3]})
    else:
        svd_pre.update({line[2]: {line[1]: line[3]}})

In [57]:
import numpy as np
import random
w=np.asarray([1, 1, 1, 1])
episilon=0.1
step=0.001
loop=10
for i in range(loop):
    error=0
    num=0
    for movie in r:
        n=len(r[movie])
        loss=0
        dw=[0, 0, 0, 0]
        for user in r[movie]:
            x=[]
            if user in content_sim and movie in content_sim[user]: x.append(content_sim[user][movie])
            else: x.append(0)            
            if user in tag_sim and movie in tag_sim[user]: x.append(tag_sim[user][movie])
            else: x.append(0)            
            if movie in cf_pre and user in cf_pre[movie]: x.append(cf_pre[movie][user])
            else: x.append(2.5)                
            if movie in svd_pre and user in svd_pre[movie]: x.append(svd_pre[movie][user])
            else: x.append(2.5)
          
            x=np.asarray(x)
            dw=dw+(np.dot(w, x)-r[movie][user])*x+w
            loss=loss+(np.dot(w, x)-r[movie][user])**2
        error=error+loss
        num=num+n
        loss=loss/n
        dw=dw/n
        w=w-step*dw
        # print('loss:', loss)
    error=error/num
    print('No.', i, 'trained successfully!', 'The main error is: ', error)
    if error<episilon:
        break

No. 0 trained successfully! The main error is:  0.6119945616854807
No. 1 trained successfully! The main error is:  0.30239998623949244
No. 2 trained successfully! The main error is:  0.30239979240509846
No. 3 trained successfully! The main error is:  0.3023997924052048
No. 4 trained successfully! The main error is:  0.30239979240520487
No. 5 trained successfully! The main error is:  0.30239979240520487
No. 6 trained successfully! The main error is:  0.30239979240520487
No. 7 trained successfully! The main error is:  0.30239979240520487
No. 8 trained successfully! The main error is:  0.30239979240520487
No. 9 trained successfully! The main error is:  0.30239979240520487


In [58]:
print(w)

[1.28231347e-04 1.87106411e-06 3.10075898e-01 6.70507815e-01]


In [59]:
for line in train_data.itertuples():    
    user=line[1]
    movie=line[2]
    rate=line[3]
    x=[]
    if movie in content_sim and user in content_sim[movie]:
        x.append(content_sim[movie][user])
    else:
        x.append(0)
            
    if movie in tag_sim and user in tag_sim[movie]:
        x.append(tag_sim[movie][user])
    else:
        x.append(0)
                
    if movie in cf_pre and user in cf_pre[movie]:
        x.append(cf_pre[movie][user])
    else:
        x.append(2.5)
                
    if movie in svd_pre and user in svd_pre[movie]:
        x.append(svd_pre[movie][user])
    else:
        x.append(2.5)
    x=np.asarray(x)
    prediction=np.dot(w, x)
    print(user, '\t', movie, '\t', rate, '\t', prediction)

1 	 1 	 4.0 	 2.9013866216320987
1 	 3 	 4.0 	 4.048918734276321
1 	 6 	 4.0 	 4.301457980521397
1 	 47 	 5.0 	 4.781041447540014
1 	 50 	 5.0 	 3.1689493880152546
1 	 70 	 3.0 	 3.4350609975217234
1 	 101 	 5.0 	 4.828605379537617
1 	 110 	 4.0 	 3.1066668121001006
1 	 151 	 5.0 	 4.306327528004291
1 	 157 	 5.0 	 4.305978735783211
1 	 163 	 5.0 	 3.839282942828221
1 	 216 	 5.0 	 4.568602086892935
1 	 223 	 3.0 	 3.9641643436506664
1 	 231 	 5.0 	 4.291642405540201
1 	 235 	 4.0 	 4.285004450260741
1 	 260 	 5.0 	 4.854187006373769
1 	 296 	 3.0 	 4.295271466640822
1 	 316 	 3.0 	 3.812495279565141
1 	 333 	 5.0 	 4.654019274044997
1 	 349 	 4.0 	 4.174280172412769
1 	 356 	 4.0 	 4.8470053093104735
1 	 362 	 5.0 	 4.506140267340788
1 	 367 	 4.0 	 4.126233915030209
1 	 423 	 3.0 	 3.495826576021547
1 	 441 	 4.0 	 4.224648531587656
1 	 457 	 5.0 	 4.867791754906799
1 	 480 	 4.0 	 4.074927010803078
1 	 500 	 3.0 	 3.6154558105991614
1 	 527 	 5.0 	 4.744100560805052
1 	 543 	 4.0 	 

2 	 91529 	 3.5 	 4.24470799757948
2 	 91658 	 2.5 	 3.079748733360014
2 	 99114 	 3.5 	 2.6964480540361797
2 	 106782 	 5.0 	 2.6751067833214313
2 	 109487 	 3.0 	 3.520180997241469
2 	 112552 	 4.0 	 4.0486192685990225
2 	 114060 	 2.0 	 2.779796109163858
2 	 115713 	 3.5 	 3.6858872806272243
2 	 122882 	 5.0 	 4.179405870917904
2 	 131724 	 5.0 	 3.348042306076816
3 	 31 	 0.5 	 1.3788250176312984
3 	 527 	 0.5 	 0.43139144196833623
3 	 647 	 0.5 	 1.5668326561488437
3 	 688 	 0.5 	 1.86563526151047
3 	 720 	 0.5 	 1.584277669997855
3 	 849 	 5.0 	 3.613566779355413
3 	 914 	 0.5 	 1.5915199832462128
3 	 1093 	 0.5 	 1.2094239462937992
3 	 1124 	 0.5 	 1.377918655074054
3 	 1263 	 0.5 	 1.5573863114131266
3 	 1272 	 0.5 	 1.6625416095984829
3 	 1275 	 3.5 	 3.15983213621538
3 	 1302 	 0.5 	 1.312492644632484
3 	 1371 	 3.0 	 2.678249181099228
3 	 1587 	 4.5 	 3.247167450385978
3 	 2018 	 0.5 	 1.3406095593399623
3 	 2080 	 0.5 	 1.2675479535867509
3 	 2090 	 0.5 	 1.3846959815734554

6 	 291 	 3.0 	 3.129646778916707
6 	 292 	 4.0 	 2.514716785986899
6 	 293 	 3.0 	 4.021060072412089
6 	 302 	 3.0 	 3.112210619462141
6 	 303 	 3.0 	 3.0513665229299507
6 	 312 	 1.0 	 1.5226009231455875
6 	 315 	 4.0 	 3.517494943625833
6 	 316 	 5.0 	 4.092815153573196
6 	 330 	 3.0 	 3.0919351562619486
6 	 333 	 5.0 	 4.29063958128985
6 	 337 	 5.0 	 4.140921017017619
6 	 340 	 4.0 	 3.696967319087721
6 	 342 	 3.0 	 3.2601950582382893
6 	 343 	 3.0 	 3.050608608770402
6 	 344 	 3.0 	 3.607428155614508
6 	 347 	 3.0 	 3.2167233549155334
6 	 348 	 3.0 	 2.991453749075443
6 	 353 	 4.0 	 3.3570525907750373
6 	 355 	 4.0 	 3.4896433523412944
6 	 357 	 3.0 	 3.235344468123327
6 	 360 	 3.0 	 3.0471293761138467
6 	 361 	 4.0 	 3.5341767919680223
6 	 366 	 3.0 	 3.3498786061854178
6 	 370 	 4.0 	 3.5825985830871065
6 	 371 	 3.0 	 3.4748835173984602
6 	 377 	 5.0 	 4.555203808051384
6 	 382 	 3.0 	 3.239694850174856
6 	 405 	 3.0 	 2.6440887207758657
6 	 410 	 3.0 	 2.1484780923911133
6

8 	 592 	 3.0 	 3.178014384862878
8 	 593 	 4.0 	 4.4446505491086015
8 	 597 	 3.0 	 3.1847483064949857
9 	 41 	 3.0 	 3.192659241997724
9 	 187 	 3.0 	 3.3958442831439175
9 	 223 	 4.0 	 3.7170836805191683
9 	 371 	 3.0 	 2.9648018839399395
9 	 627 	 3.0 	 3.08782475671752
9 	 922 	 4.0 	 3.91971413651526
9 	 923 	 5.0 	 4.236151327883986
9 	 1037 	 2.0 	 2.3600410131136345
9 	 1095 	 4.0 	 3.63294656512548
9 	 1198 	 5.0 	 4.400730806818144
9 	 1270 	 5.0 	 4.319056986022518
9 	 1674 	 4.0 	 3.473888666831657
9 	 1987 	 2.0 	 2.5751187764463705
9 	 2011 	 3.0 	 3.1739948404098595
9 	 2012 	 4.0 	 3.5683757258796467
9 	 2023 	 3.0 	 3.108939872336382
9 	 2300 	 5.0 	 4.144364870068214
9 	 2877 	 3.0 	 2.9633541321109913
9 	 2901 	 2.0 	 2.455250699426127
9 	 3173 	 2.0 	 2.4692646136989356
9 	 3328 	 3.0 	 3.402330641996818
9 	 3735 	 3.0 	 3.2174232545916395
9 	 4131 	 1.0 	 1.9739164893519379
9 	 4558 	 1.0 	 2.000939061225618
9 	 4993 	 5.0 	 4.214403758581979
9 	 5218 	 4.0 	 3.50

16 	 2858 	 3.5 	 2.3952553441671762
16 	 2959 	 3.5 	 2.5268172368411204
16 	 3000 	 4.0 	 3.709336782626636
16 	 3022 	 3.5 	 3.515672205207167
16 	 3108 	 4.0 	 3.577975737450873
16 	 3174 	 3.5 	 3.4245147245911958
16 	 3429 	 3.5 	 3.8716986636104602
16 	 3683 	 4.0 	 3.881962437462079
16 	 3741 	 4.0 	 3.6243887507894303
16 	 3949 	 4.0 	 3.724973063662854
16 	 4002 	 3.5 	 3.5569206620239058
16 	 4011 	 3.0 	 3.5193932490285484
16 	 4226 	 4.5 	 2.52088509209564
16 	 4878 	 3.5 	 2.477927785643455
16 	 4973 	 4.0 	 3.9167793374359636
16 	 4993 	 3.5 	 2.429733850333746
16 	 5618 	 4.5 	 3.977588747097589
16 	 5782 	 4.0 	 3.837291919361115
16 	 5952 	 3.5 	 2.3624528743431252
16 	 5971 	 4.0 	 3.7611965817348327
16 	 6350 	 3.5 	 3.6965499496131375
16 	 7099 	 3.5 	 3.607350509517632
16 	 7153 	 3.5 	 2.394830725576439
16 	 7361 	 3.5 	 2.6009832764417253
16 	 27611 	 4.0 	 3.5770141644505973
16 	 27773 	 4.0 	 3.7490798429664114
16 	 48394 	 4.0 	 3.046086157637172
16 	 58559 	

...

610 	 157296 	 4.0 	 3.7789348887040966
610 	 158238 	 5.0 	 4.722451858657801
610 	 158721 	 3.5 	 3.104609699717251
610 	 158872 	 3.5 	 3.4427144017964615
610 	 158956 	 3.0 	 2.799076243797195
610 	 159093 	 3.0 	 2.890106455482914
610 	 160080 	 3.0 	 2.9399043151213315
610 	 160341 	 2.5 	 2.499246761289183
610 	 160527 	 4.5 	 3.7163679662943663
610 	 160571 	 3.0 	 2.8066254744526162
610 	 160836 	 3.0 	 2.79970664602772
610 	 161582 	 4.0 	 3.838423414407759
610 	 161634 	 4.0 	 3.8230029284942475
610 	 162350 	 3.5 	 3.436418988439341
610 	 163937 	 3.5 	 3.1131726050425064
610 	 163981 	 3.5 	 3.120310647107666
610 	 164179 	 5.0 	 4.834979446179081
610 	 166528 	 4.0 	 3.9687044870197394
610 	 166534 	 4.0 	 4.003817597180815
610 	 168248 	 5.0 	 4.822698652854641
610 	 168250 	 5.0 	 4.713430151500787
610 	 168252 	 5.0 	 4.896348239050345
610 	 170875 	 3.0 	 2.865177786940057
4 	 106 	 4.0 	 3.186268953288436
4 	 759 	 3.0 	 3.094517316998233
4 	 3851 	 5.0 	 3.524894611

233 	 78103 	 2.5 	 2.649447302574176
233 	 96020 	 3.0 	 2.920197260020311
233 	 118198 	 2.5 	 2.6481421436758286
233 	 183897 	 3.5 	 3.2971786370220295
234 	 1495 	 2.0 	 2.7553288842633927
234 	 1582 	 3.0 	 3.0401518105771803
234 	 1826 	 2.0 	 2.923954647512181
244 	 3214 	 5.0 	 3.973332441471481
263 	 988 	 3.0 	 2.9500795117739056
263 	 7879 	 3.5 	 3.0710414426336614
266 	 476 	 1.0 	 2.220570173116832
268 	 869 	 3.0 	 2.98767811450454
274 	 1592 	 2.5 	 2.4938755040986176
274 	 2042 	 3.5 	 3.198224855503195
274 	 2152 	 2.0 	 2.3071985652928264
274 	 2170 	 3.0 	 2.8846734131737204
274 	 4143 	 3.5 	 3.0588268605068194
274 	 4683 	 4.0 	 3.74017277296583
274 	 5588 	 3.5 	 3.0702656205374126
274 	 7368 	 3.5 	 3.0524314273184014
274 	 8604 	 2.0 	 2.3536233102179915
274 	 26523 	 3.5 	 3.0299026368924125
274 	 26736 	 4.5 	 3.5003914499626765
274 	 27311 	 3.5 	 3.430728906713436
274 	 27618 	 3.5 	 3.300627702172087
274 	 31909 	 4.0 	 3.2829136122254097
274 	 33132 	 1.

418 	 90374 	 4.5 	 3.861277038513997
418 	 156387 	 4.0 	 3.6287704002052856
419 	 4093 	 1.5 	 2.8264198149556217
419 	 71131 	 5.0 	 4.252868901344935
419 	 85394 	 3.0 	 3.104007837673347
419 	 90943 	 5.0 	 3.4676596618846833
420 	 6210 	 3.0 	 3.077490376072742
420 	 8577 	 4.0 	 3.228846921064166
420 	 26693 	 4.0 	 3.2339033817768614
424 	 4845 	 3.0 	 3.0957923495836503
425 	 6300 	 4.0 	 3.7296130215144268
438 	 2170 	 3.0 	 2.8300368398490225
438 	 4453 	 3.5 	 3.022880109007664
438 	 4743 	 3.5 	 3.451934235858312
438 	 5787 	 2.5 	 2.7307790651567245
438 	 6992 	 3.5 	 3.2644442097424458
438 	 7192 	 3.0 	 2.942049294188993
438 	 8183 	 3.5 	 3.297470152023988
448 	 870 	 1.0 	 1.4390990920223743
448 	 1839 	 3.0 	 2.7832366413953764
448 	 2521 	 3.0 	 2.9414362319312852
448 	 2523 	 2.0 	 1.9557548038437464
448 	 2534 	 0.5 	 0.5552707116276332
448 	 2887 	 1.0 	 1.4825354370314
448 	 3276 	 1.0 	 1.4909577105854501
448 	 3390 	 2.0 	 2.1395635692972497
448 	 3596 	 1.0 	

567 	 102800 	 2.5 	 2.3451643069370345
567 	 102880 	 0.5 	 0.8497183147069527
567 	 108945 	 1.5 	 1.4780978151193787
567 	 109897 	 0.5 	 1.5550636766599637
567 	 110603 	 0.5 	 1.5818893341683857
567 	 112421 	 4.5 	 3.730543981461347
567 	 122246 	 0.5 	 1.575577234896519
567 	 122888 	 0.5 	 1.5799707201270596
567 	 132335 	 1.0 	 1.775253737541037
567 	 135288 	 3.0 	 2.523323549284816
567 	 141668 	 0.5 	 1.5786921295293148
567 	 142507 	 2.5 	 2.340097920224691
567 	 151315 	 0.5 	 0.9426239836796555
567 	 152063 	 0.5 	 1.5659069978425886
567 	 152071 	 1.5 	 1.9478554543698787
567 	 156605 	 4.5 	 3.0756768269908648
567 	 170945 	 3.5 	 2.710387220403918
567 	 176419 	 3.0 	 2.779598745164018
567 	 179491 	 1.0 	 1.7764988744723034
567 	 179749 	 1.5 	 1.9558976531919245
567 	 183897 	 3.5 	 3.049058681066777
571 	 1980 	 2.0 	 2.466639862955371
571 	 2740 	 1.0 	 2.3020808790879803
571 	 2868 	 1.0 	 1.8341790722760734
571 	 2982 	 2.0 	 2.4731341449892286
572 	 3073 	 5.0 

603 	 2008 	 3.0 	 2.8022504355056945
603 	 2068 	 5.0 	 4.803042211205737
603 	 2131 	 4.0 	 3.8810001364201128
603 	 2417 	 1.0 	 1.5313306286948882
603 	 2435 	 4.0 	 3.4439331089908904
603 	 2892 	 1.0 	 1.549435342098815
603 	 2972 	 5.0 	 4.083388738455567
603 	 3165 	 4.0 	 3.9277444442839564
603 	 3167 	 4.0 	 3.9933903269433166
603 	 3183 	 3.0 	 2.803262948748486
603 	 3431 	 1.0 	 0.9511161131168999
603 	 3473 	 5.0 	 4.134325337200452
603 	 3531 	 5.0 	 4.075335783878952
603 	 3637 	 5.0 	 4.062563270637206
603 	 3787 	 3.0 	 2.976848127063353
603 	 3792 	 5.0 	 4.07409691360447
603 	 4420 	 3.0 	 2.8141983461279407
603 	 4422 	 4.0 	 3.472985228009482
603 	 4695 	 4.0 	 3.4336853087834758
603 	 4743 	 3.0 	 2.7768836976343443
603 	 4766 	 4.0 	 3.432013533631533
603 	 4784 	 3.0 	 2.774874506352911
604 	 117 	 3.0 	 3.1052946429086177
604 	 285 	 4.0 	 3.285259927894387
604 	 636 	 3.0 	 3.1239521689939735
605 	 27036 	 3.0 	 2.863170681268686
605 	 27251 	 1.0 	 2.1604531